In [ ]:
import pandas as pd
import joblib
import numpy as np

In [25]:
# Pipeline laden
model_pipeline = joblib.load("../../data/model_pipeline.joblib")

In [26]:
# Data inladen
df_deploy = pd.read_csv("../../data/churndata_deployment.csv", sep=",", decimal=".")

X_deploy = df_deploy.copy()

In [27]:
# Kans voorspellen
y_prob = model_pipeline.predict_proba(X_deploy)[:, 1]

In [28]:
# Offer tarrief 2 berekenen

# Winstvergelijkingen
W1 = 0.1 * X_deploy["Frequency of SMS"] + 0.3 * X_deploy["Seconds of Use"]
W2 = 0.07 * X_deploy["Frequency of SMS"] + 0.2 * X_deploy["Seconds of Use"]

thresholds = (W1 - W2) / (W1 - 0.75 * W2)

# Beslissing
offer_tarief2 = (y_prob >= thresholds).astype(int)

In [29]:
# Output wegschrijven
output = pd.DataFrame({
    "Tarief2_aanbieden": offer_tarief2
})

output.to_csv("../../data/deployment_output.csv", index=False)

In [33]:
# Winst omzetten naar per maand
sms_pm = X_deploy["Frequency of SMS"] / 9
min_pm = (X_deploy["Seconds of Use"] / 60) / 9  # seconden -> minuten -> per maand

W1_pm = 0.1 * sms_pm + 0.3 * min_pm
W2_pm = 0.07 * sms_pm + 0.2 * min_pm

# Verwachte winst per klant per maand
E_tarief1 = (1 - y_prob) * W1_pm
E_tarief2 = (1 - 0.75 * y_prob) * W2_pm

# Totale verwachte winst 
totale_winst = E_tarief2[offer_tarief2 == 1].sum() + E_tarief1[offer_tarief2 == 0].sum()


print("Aantal klanten tarief 2:", int(offer_tarief2.sum()))
print("Geschatte maandelijkse winst:", round(float(totale_winst), 2))

Aantal klanten tarief 2: 18
Geschatte maandelijkse winst: 963.03
